In [12]:
import os
from test import Decoder, Encoder, LitAutoEncoder

import dotenv
import hydra
import lightning as L
import omegaconf
import torch
import torch.utils.data as data
import yaml
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import wandb

dotenv.load_dotenv(dotenv_path="../.env")
wandb.login(key=os.environ["WANDB_API_KEY"])

with hydra.initialize(version_base=None, config_path="./conf-2"):
    cfg = hydra.compose(config_name="config")

Exception in thread Exception in threading.excepthook:
Exception ignored in thread started by: <bound method Thread._bootstrap of <Thread(Thread-27 (_run_job), stopped 129177034413632)>>
Traceback (most recent call last):
  File "/home/minami/.pyenv/versions/3.11.4/lib/python3.11/threading.py", line 995, in _bootstrap
    self._bootstrap_inner()
  File "/home/minami/.pyenv/versions/3.11.4/lib/python3.11/threading.py", line 1040, in _bootstrap_inner
    self._invoke_excepthook(self)
  File "/home/minami/.pyenv/versions/3.11.4/lib/python3.11/threading.py", line 1352, in invoke_excepthook
    local_print("Exception in threading.excepthook:",
  File "/home/minami/lip2sp_pytorch/.venv/lib/python3.11/site-packages/ipykernel/iostream.py", line 604, in flush
    self.pub_thread.schedule(self._flush)
  File "/home/minami/lip2sp_pytorch/.venv/lib/python3.11/site-packages/ipykernel/iostream.py", line 267, in schedule
    self._event_pipe.send(b"")
  File "/home/minami/lip2sp_pytorch/.venv/lib/pyt

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/home/minami/lip2sp_pytorch/.venv/lib/python3.11/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [13]:
def main():
    torch.set_float32_matmul_precision("medium")

    transform = transforms.ToTensor()
    train_set = datasets.MNIST(
        root="MNIST", download=True, train=True, transform=transform
    )
    test_set = datasets.MNIST(
        root="MNIST", download=True, train=False, transform=transform
    )

    train_set_size = int(len(train_set) * 0.8)
    valid_set_size = len(train_set) - train_set_size

    seed = torch.Generator().manual_seed(42)
    train_set, valid_set = data.random_split(
        train_set, [train_set_size, valid_set_size], generator=seed
    )

    train_loader = DataLoader(
        train_set,
        num_workers=cfg["app"]["num_workers"],
        batch_size=cfg["app"]["batch_size"],
    )
    valid_loader = DataLoader(
        valid_set,
        num_workers=cfg["app"]["num_workers"],
        batch_size=cfg["app"]["batch_size"],
    )
    test_loader = DataLoader(test_set, num_workers=cfg["app"]["num_workers"])

    autoencoder = LitAutoEncoder(Encoder(cfg), Decoder(cfg), cfg)

    wandb_logger = WandbLogger(
        project="pytorch-lightning-test",
        # name="test",
        log_model=True,
        config=omegaconf.OmegaConf.to_container(
            cfg, resolve=True, throw_on_missing=True
        ),
        settings=wandb.Settings(start_method="thread"),
        # group="group-test-sweep-1",
    )

    trainer = L.Trainer(
        default_root_dir="./",
        max_epochs=20,
        callbacks=[
            EarlyStopping(monitor="val_loss", mode="min", patience=3),
            ModelCheckpoint(monitor="val_loss", mode="min", save_last=True),
        ],
        # fast_dev_run=10,
        limit_train_batches=1.0,
        limit_val_batches=1.0,
        limit_test_batches=1.0,
        profiler=None,
        logger=wandb_logger,
    )
    trainer.fit(
        model=autoencoder,
        train_dataloaders=train_loader,
        val_dataloaders=valid_loader,
    )

In [14]:
with open("sweep_grid.yaml", "r") as f:
    sweep_cfg = f.read()
sweep_cfg = yaml.safe_load(sweep_cfg)
if 'command' in sweep_cfg.keys():
    del sweep_cfg["command"]
sweep_id = wandb.sweep(sweep=sweep_cfg)
wandb.agent(sweep_id, function=main)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: rqhvpmwb
Sweep URL: https://wandb.ai/minami373/pytorch-lightning-test/sweeps/rqhvpmwb


wandb: Ctrl + C detected. Stopping sweep.
